# Optimal neighborhood size selection

In [ ]:
import copy
import os

import matplotlib.pyplot as plt
import numpy as np
from scipy import spatial
from tqdm.auto import trange

In [ ]:
def show_point_cloud(X, n=None, elev=0, azim=0, **kwargs):
    """Show and return 3-D subplot of a given point cloud. Colorization
    is performed by converting normals to corresponding RGB values
    by mapping them considering the RGB cube.
    
    Parameters
    ----------
    X : numpy.ndarray
        The point cloud of shape (N, 3), N is the number of points
    n : numpy.ndarray, optional
        The unit normals of shape (N, 3), where N is the number of
        points in the point cloud. If not given, point cloud will be
        colorized considering the values of the x-axis
    elev : float, optional
        The elevation angle in degrees rotates the camera above the
        plane of the vertical axis, with a positive angle corresponding
        to a location above that plane
    azim : float, optional
        The azimuthal angle in degrees rotates the camera about the
        vertical axis, with a positive angle corresponding to a
        right-handed rotation
    kwargs : dict, optional
        Additional keyword arguments for the scatter plotting function

    Returns
    -------
    tuple
        Figure and a 3-D subplot within
    """
    if isinstance(n, np.ndarray):
        n = np.divide(n, np.tile(np.expand_dims(
            np.sqrt(np.sum(np.square(n), axis=1)), axis=1
        ), [1, 3]))
        c = (127.5 + 127.5 * n) / 255
        cbar = True
        alpha = 1.
    else:
        c = 'k'
        cbar = False
        alpha = 0.05
    fig = plt.figure(figsize=(5, 5))
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*X.T, s=0.25, c=c, alpha=alpha, **kwargs)
    if cbar:
        cbar = fig.colorbar(s, ax=ax, pad=0, shrink=0.5)
        cbar.solids.set(alpha=1)
    ax.set_box_aspect([1, 1, 1])
    ax.axis('off')
    ax.view_init(elev=elev, azim=azim, vertical_axis='y')
    return fig, ax

In [ ]:
def estimate_normals(X, k):
    """Return the unit normals by fitting local tangent plane at each
    point in the point cloud
    
    Ref: Hoppe et al., in proceedings of SIGGRAPH 1992, pp. 71-78,
         doi: 10.1145/133994.134011
    
    Parameters
    ----------
    X : numpy.ndarray
        The point cloud of shape (N, 3), N is the number of points
    k : int or numpy.ndarray
        The number of nearest neighbors of a local neighborhood around
        a current query point
    
    Returns
    -------
    numpy.ndarray
        The unit normals of shape (N, 3), where N is the number of
        points in the point cloud
    """
    # create a kd-tree for quick nearest-neighbor lookup
    tree = spatial.KDTree(X)
    n = np.empty_like(X)
    if isinstance(k, int):
        k = np.full((X.shape[0], ), k)
    for i, p in enumerate(tqdm(X)):
        # extract the local neighborhood
        _, idx = tree.query([p], k=k[i])
        nbhd = X[idx.flatten()]
        
        # extract an eigenvector with smallest associeted eigenvalue
        nbhd_c = nbhd - np.mean(nbhd, axis=0)
        C = (nbhd_c.T @ nbhd_c) / k[i]
        U, S, VT = np.linalg.svd(C)
        n[i, :] =  U[:, np.argmin(S)]
    return n

In [ ]:
def angle_error(n_estim, n_gt, orient=True):
    """Return the angle error(s) in degrees between the estimated and
    ground truth normal vector
    
    Parameters
    ----------
    n_estim : numpy.ndsarray
        Estimated unit normals of shape (N, 3), where N is the
        number of points in the point cloud
    n_gt : numpy.ndarray
        Ground truth normals of the corresponding shape
    orient : bool, optional
        If it is set to True, orientation of the normals is taken
        into account. Otherwise, orientation does not matter
    
    Returns
    -------
    float or numpy.ndarray
        Angle error(s) in degrees
    """
    N = n_gt.shape[0]
    if orient:
        dot = np.sum(n_estim * n_gt, axis=1)
    else:
        dot = np.sum(np.abs(n_estim) * np.abs(n_gt), axis=1)
    dot = np.where(dot > 1, 1, dot)
    dot = np.where(dot < -1, -1, dot)
    if np.sum(dot) / N == 1:  # all normals matched -> no error
        return 0
    return np.arccos(dot) * 180 / np.pi

In [ ]:
def rms_angle_error(n_estim, n_gt, orient=False):
    """Return the root mean square angle error between estimated and
    ground truth normal vectors
    
    Parameters
    ----------
    n_estim : numpy.ndsarray
        Estimated unit normals of shape (N, 3), where N is the
        number of points in the point cloud
    n_gt : numpy.ndarray
        Ground truth normals of the corresponding shape
    orient : bool, optional
        If it is set to True, orientation of the normals is taken
        into account. Otherwise, orientation does not matter
    
    Returns
    -------
    float
        Root mean square angle error in degrees
    """
    err = angle_error(n_estim, n_gt, orient)
    return np.sqrt(np.mean(err ** 2))

In [ ]:
def get_opt_nn(X, scale, step=1, solver='eigenentropy'):
    """Calculate optimal neighborhood size for each point in the
    point cloud

    Ref: Weinmann et al., in ISPRS Annals 2014, pp. 181–188
         doi: 10.5194/isprsannals-II-3-181-2014

    Parameters
    ----------
    X : numpy.ndarray
        Unstructured point-cloud data of (n_points, 3) shape, where
        n_points is the number of points in (x, y, z) coordinate system
    k_scale : array-like
        Range of candidates for number of neighbors for construction of
        local neighborhood
    step : int, optional
        Scale step
    solver : string, optional
        Minimization of the loss function constructed either as Shannon
        `entropy` or `eigenentropy`

    Returns
    -------
    numpy.ndarray
        Optimal umber of neighbors for each point of the point cloud
    """
    n_points, n_dims = X.shape
    k_min, k_max = scale
    k_scale = np.arange(k_min, k_max+1, step, dtype='int64')
    tree = spatial.KDTree(X)
    _, idx = tree.query(X, k=k_max, workers=-1)
    opt_nn = []
    for p_idx in trange(n_points):
        entropy = []
        for k in k_scale:
            # eigenvalues from the neighborhood covariance
            nbhd = X[idx[p_idx, :k]]
            nbhd_c = nbhd - np.mean(nbhd, axis=0)
            cov = np.cov(nbhd_c, rowvar=False)
            e, _ = np.linalg.eig(cov)
            e = np.maximum(e, 1e-8)  # for stability sake
            e = e[np.argsort(e)[::-1]]  # sort in descending order
            e /= np.sum(e)  # normalization
            # entropy
            if solver == 'entropy':
                L = (e[0] - e[1]) / e[0]  # linearity
                P = (e[1] - e[2]) / e[0]  # planarity
                S = e[2] / e[0]  # scattering
                entropy.append(
                    - L * np.log(L)
                    - P * np.log(P)
                    - S * np.log(S)
                )
            elif solver == 'eigenentropy':
                entropy.append(
                    - e[0] * np.log(e[0])
                    - e[1] * np.log(e[1])
                    - e[2] * np.log(e[2])
                )
            else:
                raise NotImplementedError()
        opt_nn.append(k_scale[np.argmin(entropy)])
    return opt_nn

In [ ]:
bunny = os.path.join('data', 'bunny', 'bunny100k')
X = np.loadtxt(f'{bunny}.xyz')
n = np.loadtxt(f'{bunny}.normals')
fig, ax = show_point_cloud(X, n)

In [ ]:
scale = (10, 100)
step = 1
opt_nn = get_opt_nn(X, scale, step, solver='eigenentropy')
n_opt = estimate_normals(X, opt_nn)
theta = rms_angle_error(n_opt, n)
theta 